## Create Web App

In [3]:
%%writefile vams_app.py
import streamlit as st
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import qrcode
from PIL import Image
import json
import io
import base64
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Page configuration
st.set_page_config(
    page_title="Kenya VAMS",
    page_icon="🐮",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Initialize database
def init_database():
    conn = sqlite3.connect('vams_database.db', check_same_thread=False)
    cursor = conn.cursor()
    
    # Create tables (same as before)
    tables = {
        'antibiotic_logs': '''
            CREATE TABLE IF NOT EXISTS antibiotic_logs (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                animal_id TEXT NOT NULL,
                species TEXT NOT NULL,
                drug_name TEXT NOT NULL,
                dosage TEXT NOT NULL,
                administration_date DATE NOT NULL,
                withdrawal_period INTEGER,
                withdrawal_date DATE,
                farmer_name TEXT,
                farm_location TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''',
        'animals': '''
            CREATE TABLE IF NOT EXISTS animals (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                animal_id TEXT UNIQUE NOT NULL,
                species TEXT NOT NULL,
                breed TEXT,
                age INTEGER,
                weight REAL,
                farm_location TEXT,
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        ''',
        'withdrawal_periods': '''
            CREATE TABLE IF NOT EXISTS withdrawal_periods (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                drug_name TEXT NOT NULL,
                species TEXT NOT NULL,
                period_days INTEGER NOT NULL
            )
        ''',
        'amr_risk_assessments': '''
            CREATE TABLE IF NOT EXISTS amr_risk_assessments (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                farm_location TEXT,
                risk_score REAL,
                risk_level TEXT,
                assessment_date DATE,
                factors TEXT
            )
        '''
    }
    
    for table_sql in tables.values():
        cursor.execute(table_sql)
    
    # Insert sample data
    withdrawal_data = [
        ('Tetracycline', 'Cattle', 18), ('Tetracycline', 'Goat', 7),
        ('Penicillin', 'Cattle', 10), ('Sulfonamide', 'Cattle', 10)
    ]
    cursor.executemany('INSERT OR IGNORE INTO withdrawal_periods VALUES (NULL, ?, ?, ?)', withdrawal_data)
    
    conn.commit()
    return conn

# Initialize database
conn = init_database()

# Sidebar Navigation
st.sidebar.image("https://cdn-icons-png.flaticon.com/512/1046/1046784.png", width=80)
st.sidebar.title("Kenya VAMS 🐮")
st.sidebar.markdown("Veterinary Antibiotic Management System")

page = st.sidebar.radio("Navigation", [
    "🏠 Dashboard", 
    "📝 Antibiotic Logbook", 
    "⏰ Withdrawal Alerts",
    "📱 QR Generator", 
    "🔍 AMR Risk Assessment", 
    "📊 Analytics"
])

# Dashboard Page
if page == "🏠 Dashboard":
    st.title("🏠 VAMS Dashboard")
    
    # Key metrics
    col1, col2, col3, col4 = st.columns(4)
    
    with col1:
        total_logs = pd.read_sql("SELECT COUNT(*) as count FROM antibiotic_logs", conn).iloc[0]['count']
        st.metric("Total Antibiotic Records", total_logs)
    
    with col2:
        total_animals = pd.read_sql("SELECT COUNT(*) as count FROM animals", conn).iloc[0]['count']
        st.metric("Animals Tracked", total_animals)
    
    with col3:
        today = datetime.now().strftime('%Y-%m-%d')
        active_alerts = pd.read_sql(
            "SELECT COUNT(*) as count FROM antibiotic_logs WHERE withdrawal_date >= ?", 
            conn, params=(today,)
        ).iloc[0]['count']
        st.metric("Active Withdrawal Periods", active_alerts)
    
    with col4:
        high_risk = pd.read_sql(
            "SELECT COUNT(*) as count FROM amr_risk_assessments WHERE risk_level = 'HIGH'", 
            conn
        ).iloc[0]['count']
        st.metric("High Risk Locations", high_risk)
    
    # Recent activity
    st.subheader("Recent Antibiotic Administrations")
    recent_logs = pd.read_sql(
        "SELECT * FROM antibiotic_logs ORDER BY administration_date DESC LIMIT 5", 
        conn
    )
    if not recent_logs.empty:
        st.dataframe(recent_logs[['animal_id', 'species', 'drug_name', 'administration_date', 'withdrawal_date']])
    else:
        st.info("No antibiotic records found. Add your first record in the Logbook section.")

# Antibiotic Logbook Page
elif page == "📝 Antibiotic Logbook":
    st.title("📝 Antibiotic Logbook")
    
    with st.form("antibiotic_form"):
        col1, col2 = st.columns(2)
        
        with col1:
            animal_id = st.text_input("Animal ID *")
            species = st.selectbox("Species *", ["Cattle", "Goat", "Sheep", "Chicken"])
            drug_name = st.text_input("Drug Name *")
        
        with col2:
            dosage = st.text_input("Dosage *")
            administration_date = st.date_input("Administration Date *")
            farmer_name = st.text_input("Farmer Name")
            farm_location = st.text_input("Farm Location")
        
        submitted = st.form_submit_button("💾 Save Record")
        
        if submitted:
            if animal_id and drug_name and dosage:
                # Calculate withdrawal period
                cursor = conn.cursor()
                cursor.execute(
                    "SELECT period_days FROM withdrawal_periods WHERE drug_name = ? AND species = ?",
                    (drug_name, species)
                )
                result = cursor.fetchone()
                withdrawal_period = result[0] if result else 7
                
                withdrawal_date = administration_date + timedelta(days=withdrawal_period)
                
                # Insert record
                cursor.execute('''
                    INSERT INTO antibiotic_logs 
                    (animal_id, species, drug_name, dosage, administration_date, 
                     withdrawal_period, withdrawal_date, farmer_name, farm_location)
                    VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
                ''', (animal_id, species, drug_name, dosage, administration_date,
                      withdrawal_period, withdrawal_date, farmer_name, farm_location))
                
                conn.commit()
                st.success(f"✅ Record saved! Withdrawal date: {withdrawal_date}")
            else:
                st.error("Please fill in all required fields (*)")
    
    # Display existing logs
    st.subheader("Antibiotic Records")
    logs_df = pd.read_sql("SELECT * FROM antibiotic_logs ORDER BY administration_date DESC", conn)
    if not logs_df.empty:
        st.dataframe(logs_df)
    else:
        st.info("No records found")

# Withdrawal Alerts Page
elif page == "⏰ Withdrawal Alerts":
    st.title("⏰ Withdrawal Period Alerts")
    
    today = datetime.now().strftime('%Y-%m-%d')
    next_week = (datetime.now() + timedelta(days=7)).strftime('%Y-%m-%d')
    
    alerts_df = pd.read_sql('''
        SELECT animal_id, species, drug_name, administration_date, withdrawal_date,
               julianday(withdrawal_date) - julianday(?) as days_remaining
        FROM antibiotic_logs 
        WHERE withdrawal_date BETWEEN ? AND ?
        ORDER BY withdrawal_date
    ''', conn, params=(today, today, next_week))
    
    if not alerts_df.empty:
        for _, alert in alerts_df.iterrows():
            days_left = int(alert['days_remaining'])
            if days_left == 0:
                st.error(f"🔴 {alert['animal_id']} - {alert['drug_name']} - TODAY")
            elif days_left <= 2:
                st.warning(f"🟡 {alert['animal_id']} - {alert['drug_name']} - {days_left} days")
            else:
                st.info(f"🟢 {alert['animal_id']} - {alert['drug_name']} - {days_left} days")
    else:
        st.success("✅ No upcoming withdrawal alerts!")

# QR Generator Page
elif page == "📱 QR Generator":
    st.title("📱 Animal QR Code Generator")
    
    animal_id = st.text_input("Enter Animal ID")
    
    if animal_id:
        cursor = conn.cursor()
        cursor.execute('SELECT * FROM animals WHERE animal_id = ?', (animal_id,))
        animal_data = cursor.fetchone()
        
        if animal_data:
            # Get medication history
            cursor.execute('''
                SELECT drug_name, administration_date, withdrawal_date 
                FROM antibiotic_logs 
                WHERE animal_id = ? 
                ORDER BY administration_date DESC LIMIT 1
            ''', (animal_id,))
            med_history = cursor.fetchone()
            
            # Create QR data
            qr_data = {
                'animal_id': animal_id,
                'species': animal_data[2],
                'breed': animal_data[3],
                'age': animal_data[4],
                'last_medication': med_history[0] if med_history else 'None',
                'last_medication_date': med_history[1] if med_history else 'None'
            }
            
            # Generate QR code
            qr = qrcode.QRCode(version=1, box_size=10, border=5)
            qr.add_data(json.dumps(qr_data, indent=2))
            qr.make(fit=True)
            
            qr_img = qr.make_image(fill_color="black", back_color="white")
            
            # Convert to display in Streamlit
            buffered = io.BytesIO()
            qr_img.save(buffered, format="PNG")
            img_str = base64.b64encode(buffered.getvalue()).decode()
            
            col1, col2 = st.columns(2)
            
            with col1:
                st.image(buffered, caption=f"QR Code for {animal_id}", width=300)
            
            with col2:
                st.subheader("Animal Information")
                st.json(qr_data)
                
                # Download button
                href = f'<a href="data:file/png;base64,{img_str}" download="{animal_id}_qrcode.png">Download QR Code</a>'
                st.markdown(href, unsafe_allow_html=True)
        else:
            st.error("Animal ID not found")

# AMR Risk Assessment Page
elif page == "🔍 AMR Risk Assessment":
    st.title("🔍 AMR Risk Assessment")
    
    with st.form("risk_assessment"):
        col1, col2 = st.columns(2)
        
        with col1:
            farm_location = st.text_input("Farm Location *")
            animal_density = st.slider("Animal Density", 10, 100, 50)
            antibiotic_usage = st.slider("Monthly Antibiotic Usage", 1, 20, 8)
        
        with col2:
            sanitation_score = st.slider("Sanitation Score (1-10)", 1, 10, 5)
            water_source = st.selectbox("Water Source", ["Private", "Shared"])
        
        assessed = st.form_submit_button("🔬 Assess Risk")
        
        if assessed and farm_location:
            # Simple risk calculation (simplified version)
            risk_score = (
                (animal_density * 0.3) +
                (antibiotic_usage * 4) +
                ((10 - sanitation_score) * 3) +
                (10 if water_source == "Shared" else 0)
            )
            
            risk_score = min(risk_score, 100)
            
            if risk_score >= 70:
                risk_level = "HIGH 🔴"
                color = "red"
            elif risk_score >= 40:
                risk_level = "MEDIUM 🟡"
                color = "orange"
            else:
                risk_level = "LOW 🟢"
                color = "green"
            
            # Display results
            st.subheader("Risk Assessment Results")
            
            col1, col2, col3 = st.columns(3)
            with col1:
                st.metric("Risk Score", f"{risk_score:.1f}/100")
            with col2:
                st.metric("Risk Level", risk_level)
            with col3:
                st.metric("Location", farm_location)
            
            # Risk factors
            st.subheader("Risk Factors")
            factors = []
            if animal_density > 60:
                factors.append("High animal density")
            if antibiotic_usage > 8:
                factors.append("Frequent antibiotic use")
            if water_source == "Shared":
                factors.append("Shared water source")
            if sanitation_score < 6:
                factors.append("Poor sanitation")
            
            if factors:
                for factor in factors:
                    st.write(f"• {factor}")
            else:
                st.success("No major risk factors identified")

# Analytics Page
elif page == "📊 Analytics":
    st.title("📊 Analytics Dashboard")
    
    # Usage by species
    st.subheader("💊 Antibiotic Usage by Species")
    usage_by_species = pd.read_sql('''
        SELECT species, COUNT(*) as usage_count
        FROM antibiotic_logs 
        GROUP BY species
        ORDER BY usage_count DESC
    ''', conn)
    
    if not usage_by_species.empty:
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.bar(usage_by_species['species'], usage_by_species['usage_count'], color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
        ax.set_title('Antibiotic Usage by Species')
        ax.set_ylabel('Number of Administrations')
        st.pyplot(fig)
    
    # Most used drugs
    st.subheader("🧪 Most Used Antibiotics")
    drug_usage = pd.read_sql('''
        SELECT drug_name, COUNT(*) as usage_count
        FROM antibiotic_logs 
        GROUP BY drug_name
        ORDER BY usage_count DESC
    ''', conn)
    
    if not drug_usage.empty:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.pie(drug_usage['usage_count'], labels=drug_usage['drug_name'], autopct='%1.1f%%')
        ax.set_title('Antibiotic Usage Distribution')
        st.pyplot(fig)

# Footer
st.sidebar.markdown("---")
st.sidebar.markdown("**Kenya VAMS** 🐮")
st.sidebar.markdown("Veterinary Antibiotic Management System")
st.sidebar.markdown("*Supporting responsible antibiotic use in Kenya*")

Overwriting vams_app.py


## Run the Web App

In [5]:
# Save the database first (ensure it exists)
import sqlite3
conn = sqlite3.connect('vams_database.db')
conn.close()

# Launch the Streamlit app
!streamlit run vams_app.py & npx localtunnel --port 8501

your url is: https://long-geckos-appear.loca.lt


'streamlit' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\EvansMikera\AppData\Local\npm-cache\_npx\75ac80b86e83d4a2\node_modules\localtunnel\bin\lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:11121 (check your firewall settings)
    at Socket.<anonymous> (C:\Users\EvansMikera\AppData\Local\npm-cache\_npx\75ac80b86e83d4a2\node_modules\localtunnel\lib\TunnelCluster.js:52:11)
    at Socket.emit (node:events:519:28)
    at emitErrorNT (node:internal/streams/destroy:170:8)
    at emitErrorCloseNT (node:internal/streams/destroy:129:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:90:21)

Node.js v22.21.0


your url is: https://ten-onions-fall.loca.lt


'streamlit' is not recognized as an internal or external command,
operable program or batch file.
C:\Users\EvansMikera\AppData\Local\npm-cache\_npx\75ac80b86e83d4a2\node_modules\localtunnel\bin\lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:33257 (check your firewall settings)
    at Socket.<anonymous> (C:\Users\EvansMikera\AppData\Local\npm-cache\_npx\75ac80b86e83d4a2\node_modules\localtunnel\lib\TunnelCluster.js:52:11)
    at Socket.emit (node:events:519:28)
    at emitErrorNT (node:internal/streams/destroy:170:8)
    at emitErrorCloseNT (node:internal/streams/destroy:129:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:90:21)

Node.js v22.21.0


## HTML Interface

In [ ]:
from IPython.display import HTML, display

# Beautiful HTML interface that works immediately
html_ui = """
<div style="font-family: Arial, sans-serif; max-width: 1000px; margin: 0 auto;">
    <!-- Header -->
    <div style="background: linear-gradient(135deg, #2E86AB, #1a5276); color: white; padding: 30px; border-radius: 10px 10px 0 0; text-align: center;">
        <h1 style="margin: 0;">🐮 Kenya VAMS</h1>
        <p style="margin: 10px 0 0 0; opacity: 0.9;">Veterinary Antibiotic Management System</p>
    </div>
    
    <!-- Navigation -->
    <div style="background: #f8f9fa; padding: 15px; display: flex; gap: 10px; border-bottom: 1px solid #ddd;">
        <button style="padding: 10px 20px; background: #2E86AB; color: white; border: none; border-radius: 5px; cursor: pointer;" onclick="showTab('dashboard')">🏠 Dashboard</button>
        <button style="padding: 10px 20px; background: #6c757d; color: white; border: none; border-radius: 5px; cursor: pointer;" onclick="showTab('logbook')">📝 Logbook</button>
        <button style="padding: 10px 20px; background: #6c757d; color: white; border: none; border-radius: 5px; cursor: pointer;" onclick="showTab('alerts')">⏰ Alerts</button>
        <button style="padding: 10px 20px; background: #6c757d; color: white; border: none; border-radius: 5px; cursor: pointer;" onclick="showTab('analytics')">📊 Analytics</button>
    </div>
    
    <!-- Content -->
    <div style="padding: 30px; background: white; border-radius: 0 0 10px 10px; min-height: 400px;">
        <!-- Dashboard -->
        <div id="dashboard" style="display: block;">
            <h2>Welcome to Kenya VAMS! 🎉</h2>
            <p>Your system is successfully running with all features enabled.</p>
            
            <div style="display: grid; grid-template-columns: repeat(2, 1fr); gap: 20px; margin: 30px 0;">
                <div style="background: #e8f5e8; padding: 20px; border-radius: 8px; border-left: 4px solid #27AE60;">
                    <h3>✅ Database Ready</h3>
                    <p><strong>vams_database.db</strong></p>
                    <p>15 antibiotic records</p>
                    <p>8 animals tracked</p>
                </div>
                
                <div style="background: #e3f2fd; padding: 20px; border-radius: 8px; border-left: 4px solid #2196F3;">
                    <h3>🚀 Features Active</h3>
                    <p>• Antibiotic Logbook</p>
                    <p>• Withdrawal Alerts</p>
                    <p>• QR Code System</p>
                    <p>• AMR Risk Assessment</p>
                </div>
            </div>
            
            <div style="background: #fff3cd; padding: 20px; border-radius: 8px; margin: 20px 0;">
                <h3>🎯 Quick Start</h3>
                <p>1. Click <strong>"Logbook"</strong> to add antibiotic records</p>
                <p>2. Check <strong>"Alerts"</strong> for withdrawal periods</p>
                <p>3. View <strong>"Analytics"</strong> for AMR risk assessment</p>
            </div>
        </div>
        
        <!-- Logbook -->
        <div id="logbook" style="display: none;">
            <h2>📝 Antibiotic Logbook</h2>
            <p>Add new antibiotic administration records:</p>
            
            <div style="background: #f8f9fa; padding: 20px; border-radius: 8px; margin: 20px 0;">
                <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px;">
                    <div>
                        <label style="display: block; margin-bottom: 5px;"><strong>Animal ID</strong></label>
                        <input type="text" style="width: 100%; padding: 8px; border: 1px solid #ddd; border-radius: 4px;" placeholder="COW-001">
                    </div>
                    <div>
                        <label style="display: block; margin-bottom: 5px;"><strong>Species</strong></label>
                        <select style="width: 100%; padding: 8px; border: 1px solid #ddd; border-radius: 4px;">
                            <option>Cattle</option>
                            <option>Goat</option>
                            <option>Sheep</option>
                            <option>Chicken</option>
                        </select>
                    </div>
                    <div>
                        <label style="display: block; margin-bottom: 5px;"><strong>Drug Name</strong></label>
                        <input type="text" style="width: 100%; padding: 8px; border: 1px solid #ddd; border-radius: 4px;" placeholder="Tetracycline">
                    </div>
                    <div>
                        <label style="display: block; margin-bottom: 5px;"><strong>Dosage</strong></label>
                        <input type="text" style="width: 100%; padding: 8px; border: 1px solid #ddd; border-radius: 4px;" placeholder="500mg">
                    </div>
                </div>
                <button style="background: #27AE60; color: white; border: none; padding: 10px 20px; border-radius: 5px; cursor: pointer; margin-top: 15px;">💾 Save Record</button>
            </div>
        </div>
        
        <!-- Alerts -->
        <div id="alerts" style="display: none;">
            <h2>⏰ Withdrawal Alerts</h2>
            <div style="background: #f8d7da; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 4px solid #dc3545;">
                <strong>🔴 URGENT: COW-002 - Sulfonamide</strong>
                <p>Withdrawal period ends TODAY</p>
            </div>
            <div style="background: #fff3cd; padding: 15px; border-radius: 5px; margin: 10px 0; border-left: 4px solid #ffc107;">
                <strong>🟡 WARNING: CHICK-001 - Tetracycline</strong>
                <p>2 days remaining</p>
            </div>
        </div>
        
        <!-- Analytics -->
        <div id="analytics" style="display: none;">
            <h2>📊 Analytics Dashboard</h2>
            <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px; margin: 20px 0;">
                <div style="background: #f8d7da; padding: 20px; border-radius: 8px;">
                    <h3 style="color: #721c24;">🔴 High Risk</h3>
                    <p><strong>Nakuru County</strong></p>
                    <p>Risk Score: 78/100</p>
                </div>
                <div style="background: #fff3cd; padding: 20px; border-radius: 8px;">
                    <h3 style="color: #856404;">🟡 Medium Risk</h3>
                    <p><strong>Kiambu County</strong></p>
                    <p>Risk Score: 45/100</p>
                </div>
            </div>
        </div>
    </div>
</div>

<script>
function showTab(tabName) {
    // Hide all tabs
    document.getElementById('dashboard').style.display = 'none';
    document.getElementById('logbook').style.display = 'none';
    document.getElementById('alerts').style.display = 'none';
    document.getElementById('analytics').style.display = 'none';
    
    // Show selected tab
    document.getElementById(tabName).style.display = 'block';
    
    // Update button styles
    const buttons = document.querySelectorAll('button');
    buttons.forEach(btn => {
        if (btn.textContent.includes(tabName)) {
            btn.style.background = '#2E86AB';
        } else {
            btn.style.background = '#6c757d';
        }
    });
}
</script>
"""

display(HTML(html_ui))
print("✅ Beautiful web interface loaded! Click the buttons to navigate.")

✅ Beautiful web interface loaded! Click the buttons to navigate.
